# TFLite Interpreter CaloryIn Capstone

## Setup

### Check working directory 

In [ ]:
import os
cwd = os.getcwd()
print(cwd)

### Insert path

In [ ]:
MODEL_PATH = r'Model/tflite_model_final/fc_model.tflite'
LABEL_PATH = r'Model/tflite_model_final/labels.txt'
DATASET_PATH = r'../Dataset/food_datasets/'

### Load the TFLite model

In [ ]:
import tensorflow as tf

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)
interpreter.allocate_tensors()

### Load labels

In [ ]:
with open(LABEL_PATH, 'r') as f:
    class_labels = f.read().splitlines()
        
print(class_labels)

### Load and preprocess the input image

In [ ]:
import numpy as np
from PIL import Image

# Load and preprocess the input image
def load_image(IMAGE_PATH):
    image = Image.open(IMAGE_PATH)
    image = image.resize((224, 224))  # Resize to match the model's input size
    image = image.convert('RGB')  # Convert to RGB if needed
    arr_image = np.array(image) / 255  # Normalize pixel values
    return image, arr_image

### Process the input image

In [ ]:
def process_input(arr_image):
    # Get input and output details from the model
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Set the input tensor with the preprocessed image
    input_data = np.expand_dims(arr_image, axis=0).astype(input_details[0]['dtype'])
    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Run the inference
    interpreter.invoke()

    # Get the output tensor
    output_data = interpreter.get_tensor(output_details[0]['index'])

    # Process the output (e.g., get the predicted class or probabilities)
    predicted_class = np.argmax(output_data)
    
    return predicted_class

### Print prediction

In [ ]:
def print_prediction(predicted_class_index):
    # Map the index to the class label
    predicted_class_label = class_labels[predicted_class_index]

    # Display image 
    display(image)

    # Print the predicted class label
    print("Predicted index:",predicted_class_index, "-> Predicted class:", predicted_class_label, "\n\n")


## Single image

In [ ]:
IMAGE_PATH = r'..\Dataset\archive\Pizza-Test (23).jpeg' #Replace with your image
image, arr_image = load_image(IMAGE_PATH)
display(image)
display(arr_image)

In [ ]:
# Get the index of the predicted class with the highest probability
predicted_class_index = process_input(arr_image)

In [ ]:
print_prediction(predicted_class_index)

### Print index list of class

In [ ]:
print(predicted_class_index)

## Multiple images from folder

In [ ]:
FOLDER_IMAGE_PATH = r'..\Dataset\archive'
items = os.listdir(FOLDER_IMAGE_PATH)
print(items)

In [ ]:
for item in items:
    IMAGES_PATH = os.path.join(FOLDER_IMAGE_PATH, item)
    image, arr_image = load_image(IMAGES_PATH)
    predicted_class_index = process_input(arr_image)
    print_prediction(predicted_class_index)